# AEDDM Specific Parameters 
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(1.0, 4.5, 0.5))
count_threshold_measure=['median','max']
logical_parameter=['and','or']


In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max', 'and'),
 (3.0, 'max', 'or'),
 (3.5, 'median', 'and'),
 (3.5, 'median', 'or'),
 (3.5, 'max', 'and'),
 (3.5, 'max', 'or'),
 (4.0, 'median', 'and'),
 (4.0, 'median', 'or'),
 (4.0, 'max', 'and'),
 (4.0, 'max', 'or')]

In [4]:
len(all_combinations)

28

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20
WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)] # As drift is detectd based on three consecutive batchses,for last two batches drift cant be detected.
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)


Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('gaussian_sudden_nondrifted.csv')

In [11]:
data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,class
0,5.102913,5.511221,6.713308,2.254608,3.898497,6.006594,0.419448,1.482079,2.715893,5.702509,...,1.275115,0.469462,4.327988,2.247093,-1.142004,0.344620,8.441320,6.085484,6.143142,1
1,0.335908,0.966295,1.069843,-0.010379,0.467792,0.526575,0.368619,0.234414,0.357833,0.642754,...,0.520709,1.134835,0.487783,0.272321,0.138597,0.514048,0.616427,0.398510,0.180430,0
2,6.045675,5.426378,6.874980,3.569168,2.248690,3.344724,1.428138,1.495347,3.457193,3.134428,...,0.560366,6.467474,3.167635,0.076273,2.108656,5.216149,7.298202,3.421333,5.074158,1
3,7.675341,5.633243,5.873911,-1.716345,6.537527,7.728881,5.248266,9.532283,5.274746,-0.489477,...,-0.130820,10.280230,4.154225,7.009006,2.875988,3.902338,7.359699,1.005415,4.601388,1
4,7.028135,-2.888131,7.001985,-0.751050,2.953760,8.644294,1.275466,5.348067,6.638609,2.153154,...,3.279920,2.086240,1.535359,2.834501,2.186823,1.585804,5.038731,2.747392,4.706377,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8.086882,5.292165,7.421719,3.575924,4.083260,5.982060,0.751348,5.050999,5.971245,2.766531,...,4.322552,2.427274,4.638802,1.061286,0.648840,2.935872,7.418080,0.716601,6.446652,1
59996,4.124264,6.084786,5.870661,3.864057,1.753564,6.628186,-0.753955,5.385549,3.051964,4.061186,...,-0.367688,-0.377070,5.639479,2.182200,5.750681,4.038628,7.358345,4.431231,5.395807,1
59997,3.949366,2.586330,7.412331,-4.030309,4.852872,10.254736,-4.524299,6.344352,3.604104,3.163081,...,2.145932,1.676406,2.816040,1.477284,4.506743,4.816691,4.223846,0.721165,4.353225,1
59998,6.110675,5.856077,6.324222,3.704552,5.071043,6.348181,3.057830,5.433281,1.999805,1.543263,...,4.844653,4.361461,3.816462,1.015422,0.214616,6.059436,6.270376,0.513956,2.431868,1


In [12]:
drifted_data=pd.read_csv('gaussian_sudden_drifted.csv')

In [13]:
drifted_data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19
0,0.335300,0.614998,0.352563,0.368605,0.422216,0.306789,0.336432,0.281525,0.198037,0.511503,0.111129,0.512082,0.917936,0.607234,0.398931,0.110454,0.427349,0.317783,0.201820,0.307378
1,6.926788,4.017759,3.483361,3.066276,5.376368,7.293220,2.731916,6.058773,1.943065,3.760152,9.275068,5.171033,5.195403,6.267629,2.820670,2.347166,4.359446,8.900609,-2.172866,4.217112
2,5.586476,9.955049,6.803001,2.252187,2.626078,7.720829,4.069162,1.414253,0.349349,5.500343,7.699813,2.792029,3.308026,5.211922,-0.436737,3.255014,0.493191,5.490615,1.486799,5.911093
3,7.037762,5.900497,3.598764,3.370448,5.649035,4.825158,4.426751,9.944331,2.937494,1.517042,5.305906,2.428357,6.748053,4.328056,5.717691,3.701823,4.084876,5.683989,-2.747463,7.247130
4,7.278160,5.425232,5.919949,4.436051,4.861111,6.402653,4.425345,5.599502,3.141993,7.834311,5.263765,2.390105,4.210942,5.872883,3.371386,3.345928,5.502934,3.523283,2.060165,7.445411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.485875,0.591188,0.688523,0.602972,0.353299,-0.171085,0.394570,0.525485,0.715492,0.303763,0.428988,0.291495,1.233072,0.666615,0.448258,0.755709,0.461653,0.887536,0.254179,1.076744
5996,5.458152,6.619926,10.631114,7.559972,5.173186,9.739922,5.258557,13.195139,10.023015,12.373288,6.800868,3.168091,9.505610,3.502541,9.209186,6.654869,9.132318,13.527701,1.856816,7.538750
5997,8.677621,9.990081,5.707227,8.760084,2.356459,8.262255,4.038807,7.023272,6.536903,6.967437,2.676920,1.796819,6.817212,5.405707,10.830697,8.056248,5.672806,7.674458,8.026322,8.661300
5998,5.872419,7.370930,8.949004,8.564026,6.506088,6.736418,6.382704,3.237444,7.900552,7.967353,6.774440,5.457721,8.921901,2.189033,10.977921,9.129438,6.597944,6.537483,8.927587,4.211815


## 3. Data Preprocessing & Splitting
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [14]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding 
    # Apply MinMAx Normalization
    # Split into training  , test (validation set 1) and stream ( non-drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    
    y_positive=train_positives[label_col] 
    y_negative=train_negatives[label_col]
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    
    
    return train, test, X_positive,X_negative , stream ,y_positive ,y_negative  

In [15]:
train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'class',1,0)

### 4. Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [16]:
def autoencoder (train,epochs,val_set):
    
          
    # input layer size= # of attributes in the dataset after one-hot encoding
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded = layers.Dense(6, activation='relu')(input_layer) # Code Layer 1 , apprx 1/3 of the previous layer
    encoded=layers.Dense(2,activation='relu')(encoded) # Bottleneck  , approx 1/3 of the previous layer
    decoded=layers.Dense(6,activation='relu')(encoded)# Decode Layer 1
    decoded = layers.Dense(train.shape[1], activation='sigmoid')(decoded) # Output Layer
          
    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    # above model maps an input to its reconstruction 
   
    autoencoder.compile(optimizer='adam', loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [17]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [18]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
591/591 [==============================] - 3s 5ms/step - loss: 17.4662 - val_loss: 15.9804
Epoch 2/100
591/591 [==============================] - 1s 2ms/step - loss: 15.9920 - val_loss: 15.9793
Epoch 3/100
591/591 [==============================] - 1s 3ms/step - loss: 15.9567 - val_loss: 15.9792
Epoch 4/100
591/591 [==============================] - 2s 3ms/step - loss: 15.9646 - val_loss: 15.9791
Epoch 5/100
591/591 [==============================] - 2s 3ms/step - loss: 16.0268 - val_loss: 15.9791
Epoch 6/100
591/591 [==============================] - 2s 3ms/step - loss: 15.9828 - val_loss: 15.9791
Epoch 7/100
591/591 [==============================] - 2s 3ms/step - loss: 15.9931 - val_loss: 15.9791
Epoch 8/100
591/591 [==============================] - 1s 3ms/step - loss: 16.0285 - val_loss: 15.9791
Epoch 9/100
591/591 [==============================] - 1s 2ms/step - loss: 15.9848 - val_loss: 15.9791
Epoch 10/100
591/591 [========

591/591 [==============================] - 2s 3ms/step - loss: 15.7054 - val_loss: 15.6966
Epoch 80/100
591/591 [==============================] - 2s 3ms/step - loss: 15.7045 - val_loss: 15.6966
Epoch 81/100
591/591 [==============================] - 2s 3ms/step - loss: 15.6880 - val_loss: 15.6966
Epoch 82/100
591/591 [==============================] - 2s 3ms/step - loss: 15.6912 - val_loss: 15.6966
Epoch 83/100
591/591 [==============================] - 2s 3ms/step - loss: 15.7020 - val_loss: 15.6966
Epoch 84/100
591/591 [==============================] - 2s 3ms/step - loss: 15.7043 - val_loss: 15.6966
Epoch 85/100
591/591 [==============================] - 2s 3ms/step - loss: 15.6951 - val_loss: 15.6966
Epoch 86/100
591/591 [==============================] - 2s 3ms/step - loss: 15.7330 - val_loss: 15.6966
Epoch 87/100
591/591 [==============================] - 2s 3ms/step - loss: 15.7273 - val_loss: 15.6966
Epoch 88/100
591/591 [==============================] - 2s 3ms/step - loss: 1

591/591 [==============================] - 4s 7ms/step - loss: 0.0411 - val_loss: 0.0401
Epoch 59/100
591/591 [==============================] - 4s 7ms/step - loss: 0.0409 - val_loss: 0.0402
Epoch 60/100
591/591 [==============================] - 3s 5ms/step - loss: 0.0411 - val_loss: 0.0402
Epoch 61/100
591/591 [==============================] - 4s 7ms/step - loss: 0.0411 - val_loss: 0.0402
Epoch 62/100
591/591 [==============================] - 3s 5ms/step - loss: 0.0409 - val_loss: 0.0402
Epoch 63/100
591/591 [==============================] - 4s 7ms/step - loss: 0.0409 - val_loss: 0.0402
Epoch 64/100
591/591 [==============================] - 3s 5ms/step - loss: 0.0412 - val_loss: 0.0402
Epoch 65/100
591/591 [==============================] - 4s 7ms/step - loss: 0.0409 - val_loss: 0.0402
Epoch 66/100
591/591 [==============================] - 4s 7ms/step - loss: 0.0413 - val_loss: 0.0402
Epoch 67/100
591/591 [==============================] - 3s 5ms/step - loss: 0.0410 - val_loss: 

In [19]:
test

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,class
42000,0.050835,0.491070,0.992440,-0.224452,1.224729,0.463136,0.436940,0.328452,0.231155,0.583983,...,0.415980,0.683767,0.496750,-0.046689,0.289802,0.441804,0.847019,0.260870,0.268387,0
42001,-0.402236,-0.060298,0.230602,-0.183273,0.613884,0.621114,0.379894,0.204145,0.510970,0.890581,...,0.412858,0.405664,0.506891,0.012074,0.125762,0.493421,0.360404,0.728867,0.405936,0
42002,0.442750,0.280615,0.080240,0.091004,-0.441025,0.530508,0.380783,0.380051,0.414064,0.544097,...,0.543837,0.431445,0.265639,0.113904,0.321488,0.408750,0.366497,0.560963,0.392358,0
42003,8.259135,4.783414,6.981261,4.494583,3.372996,7.809731,2.999991,3.695592,4.823650,0.574189,...,4.244586,4.178659,1.336421,0.609191,4.448811,3.212038,5.268701,-0.444150,4.801038,1
42004,6.558045,3.965109,4.986110,3.526167,6.161957,5.739824,2.007499,2.652978,0.746049,0.501376,...,3.209521,5.721578,4.681672,5.948536,0.771566,5.864487,6.040335,0.365140,2.610868,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53995,7.202140,4.472336,8.333243,2.933322,5.280335,7.130684,0.232006,5.818234,5.239779,1.304155,...,3.257692,-0.167190,4.016159,1.238548,4.677375,4.477924,5.584874,2.384526,7.505077,1
53996,5.118851,1.399489,7.886686,1.097765,3.726823,5.426106,1.865744,4.653517,2.939199,4.694456,...,1.399290,-2.352035,-1.838180,0.367283,1.634957,0.263057,9.425868,5.716905,5.243845,1
53997,4.188361,3.429535,11.038316,5.741994,1.912404,1.745566,2.197546,3.320142,6.017018,6.387002,...,2.257219,4.559037,3.996095,1.197339,-1.412886,4.361493,6.548601,3.819951,8.156569,1
53998,-0.198354,1.004792,0.345642,-0.196843,1.532229,0.470298,0.340185,0.125852,0.318887,0.389714,...,0.213542,0.898774,0.289426,0.449683,0.274360,0.424256,0.674589,0.357691,-0.063205,0


## Processing Validation Data to compute Reconstruction Error

In [20]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [21]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['class']==1]
    test_neg_class=test[test['class']==0]
    del test_pos_class['class']
    del test_neg_class['class']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.round(np.mean(np.power(batch[i] - pred, 2)),6)
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
       
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [22]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 

### AEDDM Framwework

In [23]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.round(np.mean(np.power(test_batch[i] - pred, 2)),5)
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.round(np.mean(np.power(test_batch[i] - pred, 2)),5)
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [24]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [25]:
def compute_delay(drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values

In [26]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [27]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=100):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
drifted=drifted_data[0:1280]
batches_d=make_batches(drifted)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    

    
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.5))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d



## GRID SEARCH

In [28]:
results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 16.175791841320386 , Count Threshold: 9.0
Thresholds for Nagative AE
Batch Thresold: 0.04292659643996123 , Count Threshold: 8.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15, 17, 18, 22, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 4, 9, 12, 13, 17, 18]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19, 20, 21, 23, 24, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 19, 24, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder In


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 5, 6, 8, 14, 18, 19, 20, 24, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 11



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Poi


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15, 17, 18, 22, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 4, 9, 12, 13, 17, 18]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 8



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19, 20, 21, 23, 24, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 19, 24, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 9, 13, 14, 19, 25]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Bat


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 5, 6, 8, 14, 18, 19, 20, 24, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 11



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Poi


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 22, 25, 26, 27, 28, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 3, 9, 12, 17]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 17, 19, 21, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 12, 19]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 3



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 18, 19, 20, 22, 23, 24, 25, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [5, 9, 13, 14, 19]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Thr

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 3, 4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 23



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 2


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 19, 20, 22, 23, 25, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 6, 7, 8, 11, 12, 14, 15, 16, 18, 19, 20, 25] 

Da


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 26, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 21



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31] 

Data 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 19, 20, 22, 23, 25, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 6, 7, 8, 11, 12, 14, 15, 16, 18, 19, 20, 25] 

Da


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 26, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 21



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31] 

Data 

In [29]:
results_df_p

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.5,median,and,0,0,0,8,8,144
1,0.5,median,or,0,0,0,18,18,174
2,0.5,max,and,0,0,0,8,8,144
3,0.5,max,or,0,0,0,8,8,144
4,1.0,median,and,0,0,0,8,8,144
5,1.0,median,or,0,0,0,18,18,174
6,1.0,max,and,0,0,0,8,8,144
7,1.0,max,or,0,0,0,18,18,174
8,1.5,median,and,0,0,0,8,8,144
9,1.5,median,or,0,0,0,18,18,174


In [30]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [31]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Categorical, Real


def aeddm_function(x):
    global results_df
    row=0
    k,alpha,logical=x
    values=[k, alpha, logical]
    #results_df.iloc[row :3] = values
        
    print("***************************************************")
    print(k,alpha,logical)
    parameters=(k,alpha,logical)
    instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
    instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
    #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
    # Counts the MSE values exceeding threshold in each batch 
    exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
    exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
    count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
    count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
    print("Thresholds for Positive AE")
    #print(alpha)
    print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
    print("Thresholds for Nagative AE")
    print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
    # AEDDM Starts Here 
                
    thres_zscore_batch_pos=batch_thresh_pos
    thres_zscore_batch_neg=batch_thresh_neg
                             
                
    #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
        #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
             #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    print(" Drifted Data Processing Starts Here")
                
    score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    #value+=100
    print(col_values)
    print(values)
    values.extend(col_values)
    print(values)
    
    results_df = results_df.append(pd.Series(values, index=results_df.columns), ignore_index=True)
    print(results_df)
    
    values=[]
    col_values=[]
    print(k,alpha,logical)
    return score 



In [32]:
# Define the search space
row=0
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 1 to 4
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    #aeddm_function,
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='EI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 19.4117679632614 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.055481664808061014 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 19, 20, 22, 23, 25, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 6, 7, 8, 11, 12, 14, 15, 16, 18, 19, 20, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 8, 10, 11, 13, 15, 19, 22, 23, 24, 25, 26, 27, 29] 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 26, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 21



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 7, 12, 20, 22, 23, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Ins


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 8, 10, 11, 13, 15, 19, 22, 23, 24, 25, 26, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 5

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 8, 10, 14, 15, 16, 17, 21, 22, 23, 25, 27, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 17, 23, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 6

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 25, 26, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 7, 8, 10, 11, 13,

...

***************************************************
1.8276609909785546 max or
Thresholds for Positive AE
Batch Thresold: 17.406765626637966 , Count Threshold: 3
Thresholds for Nagative AE
Batch Thresold: 0.04770257730372933 , Count Threshold: 2
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2

***************************************************
1.8320143666095052 max or
Thresholds for Positive AE
Batch Thresold: 17.410801966724595 , Count Threshold: 3
Thresholds for Nagative AE
Batch Thresold: 0.047718237655918254 , Count Threshold: 2
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2

Best parameters: [1.8576875940670263, 'median', 'or']


In [33]:
best_params

[1.8576875940670263, 'median', 'or']

In [34]:
pd.set_option('display.max_rows', None)
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,0,8,8,144
1,3.996642,max,and,0,0,0,8,8,144
2,1.857688,median,or,0,0,0,18,18,174
3,3.462088,max,or,0,0,0,18,18,174
4,2.052085,max,or,0,0,0,18,18,174
5,3.698867,max,and,0,0,0,8,8,144
6,3.786947,median,or,0,0,0,18,18,174
7,3.309651,max,or,0,0,0,18,18,174
8,3.527571,median,or,0,0,0,18,18,174
9,1.455675,max,or,0,0,0,18,18,174


In [35]:
results_df_EI=results_df.copy()

In [36]:
results_df=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [37]:
# Define the search space
search_space = [
    Real(0.5, 4),  # Numerical parameter k with range 0.5 to 4.5
    Categorical(['median', 'max']),  # Categorical parameter alpha
    Categorical(['and', 'or'])  # Categorical parameter logical
]

# Perform the Bayesian optimization (maximization)
result = gp_minimize(
    
    lambda x: -aeddm_function(x),  # Negate the function to maximize
    dimensions=search_space,
    n_calls=20,  # Number of iterations
    acq_func='PI',  # Expected Improvement for maximization
    random_state=1  # Set a random seed for reproducibility
)

# Get the best parameters found
best_params = result.x
print("Best parameters:", best_params)

***************************************************
3.9901468382860408 median and
Thresholds for Positive AE
Batch Thresold: 19.4117679632614 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.055481664808061014 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 17, 19, 20, 22, 23, 25, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 5, 6, 7, 8, 11, 12, 14, 15, 16, 18, 19, 20, 25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 8, 10, 11, 13, 15, 19, 22, 23, 24, 25, 26, 27, 29] 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 4, 5, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 26, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 21



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 7, 12, 20, 22, 23, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Ins


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 8, 10, 11, 13, 15, 19, 22, 23, 24, 25, 26, 27, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 5

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 8, 10, 14, 15, 16, 17, 21, 22, 23, 25, 27, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 17, 23, 27]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 6

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 4, 8, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 23, 25, 26, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 7

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 7, 8, 10, 11, 13,

...

***************************************************
3.6661403626417983 median or
Thresholds for Positive AE
Batch Thresold: 19.1113573309529 , Count Threshold: 0.0
Thresholds for Nagative AE
Batch Thresold: 0.05431611973448519 , Count Threshold: 0.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14,


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2

***************************************************
1.5008294347542852 max or
Thresholds for Positive AE
Batch Thresold: 17.103735650868252 , Count Threshold: 4
Thresholds for Nagative AE
Batch Thresold: 0.04652686959761366 , Count Threshold: 3
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 17, 18, 25, 26, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 17, 19, 24, 26, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 6, 8, 10, 12, 14, 15, 17, 18, 19, 21, 24, 25, 26, 27, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 19



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 5, 8, 10, 11, 12, 13, 17, 18, 20, 21, 22, 23, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2

Best parameters: [1.8576875940670263, 'median', 'or']


In [38]:
results_df

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,3.990147,median,and,0,0,0,8,8,144
1,3.996642,max,and,0,0,0,8,8,144
2,1.857688,median,or,0,0,0,18,18,174
3,3.462088,max,or,0,0,0,18,18,174
4,2.052085,max,or,0,0,0,18,18,174
5,3.698867,max,and,0,0,0,8,8,144
6,3.786947,median,or,0,0,0,18,18,174
7,3.309651,max,or,0,0,0,18,18,174
8,3.527571,median,or,0,0,0,18,18,174
9,1.455675,max,or,0,0,0,18,18,174


In [39]:
max_value = result.fun  # Access the maximum function value
print("Maximum value:", max_value)
print("Best parameters:", result.x)

Maximum value: -174
Best parameters: [1.8576875940670263, 'median', 'or']


### TESTING CODE

In [40]:
Thresh=0.212964
count_thresh=0
logical='and'

In [41]:
drift_point=20
WnDs= 0 
WdDs=0 
DnDs=0          
delay=None     
DdDs=0   

In [42]:
def compute_delay(drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=20
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(drift_batches,drift_batches_after_drift_point)
    value=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={value}")
    
    

In [43]:
detect_drift2(avg_mse_l2_list_d,exceed_count_L2_countThresh_d, Thresh,count_thresh,logical)

 Threshold exceeds at batch : 20
[20]
Warning Level at Batch 20
 Threshold exceeds at batch : 21
[20, 21]
Warning Level at Batch 21
 Threshold exceeds at batch : 22
[20, 21, 22]
 Drift Confirmed at Batch No :  20
 Threshold exceeds at batch : 23
[20, 21, 22, 23]
 Drift Confirmed at Batch No :  21
 Threshold exceeds at batch : 24
[20, 21, 22, 23, 24]
 Drift Confirmed at Batch No :  22
 Threshold exceeds at batch : 25
[20, 21, 22, 23, 24, 25]
 Drift Confirmed at Batch No :  23
 Threshold exceeds at batch : 26
[20, 21, 22, 23, 24, 25, 26]
 Drift Confirmed at Batch No :  24
 Threshold exceeds at batch : 27
[20, 21, 22, 23, 24, 25, 26, 27]
 Drift Confirmed at Batch No :  25
 Threshold exceeds at batch : 28
[20, 21, 22, 23, 24, 25, 26, 27, 28]
 Drift Confirmed at Batch No :  26
 Threshold exceeds at batch : 29
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
 Drift Confirmed at Batch No :  27
 Threshold exceeds at batch : 30
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
 Drift Confirmed at Batch No :